# Automated_optimization_testing_Sartorious

Importing relevant packages

In [2]:
#Import relevant python packages
#%% General Imports
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt

# plotting dependencies
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

Robot initialization and checks

In [ ]:
#Import robot related packages and run setup
import pandas as pd
import time
from matplotlib import pyplot as plt
from pathlib import Path
import sys
REPOS = 'GitHub'
ROOT = str(Path().absolute()).split(REPOS)[0]
sys.path.append(f'{ROOT}{REPOS}')

from polylectric.configs.SynthesisB1 import SETUP, LAYOUT_FILE

from controllably import load_deck      # optional
load_deck(SETUP.setup, LAYOUT_FILE)     # optional

platform = SETUP
platform.mover.verbose = False

In [ ]:
#Initialization of variables for platform objects
pipette= platform.setup
deck = platform.setup.deck
balance = platform.balance
balance_deck = deck.slots['1']
source = deck.slots['2']
tip_rack = deck.slots['3']
bin = deck.slots['4']
pipette.mover.setSpeed(50)
print(balance_deck)
print(source)
print(tip_rack)
print(bin)

In [ ]:
#Check if balance is connected
balance.zero()
balance.toggleRecord(True)
time.sleep(5)
print(balance.buffer_df.iloc[-1])
balance.toggleRecord(False)

In [12]:
#Establish initial height of liquid on the source vial
pipette_name = 'rLine1000'
initial_liquid_level = 10.5


Importing files + getting parameters

In [8]:
# Change according to experiment
std = "1275"
liquid_name = 'Viscosity_std_' + std 
density = 0.8736
# Import initialisation file?
REPO = 'viscosity_liquid_transfer_Pablo'
folder = os.getcwd().split(REPO)[0]+REPO+r'\Sartorious_experiments'
df_para = pd.read_csv(folder+r'\Cleaned_Data\Viscosity_std_'+std+'_modified.csv')
df_para


,liquid,pipette,volume,aspiration_rate,dispense_rate,blow_out,delay_aspirate,delay_dispense,delay_blow_out,density,time,m,%error,Transfer_Observation,Comment,iteration,time_for_1000
0,Viscosity_std_1275,rLine1000,1000.0,28.915134,28.915134,False,10,10,0,0.8736,132.773934,0.788634,-9.725963,Incomplete aspiration and dispense,Blowout before,1,89.167931
1,Viscosity_std_1275,rLine1000,500.0,28.915134,28.915134,False,10,10,0,0.8736,97.980054,0.365575,-16.305995,Incomplete aspiration and dispense,Blowout before,1,89.167931
2,Viscosity_std_1275,rLine1000,300.0,28.915134,28.915134,False,10,10,0,0.8736,84.264670,0.202553,-22.713117,Incomplete aspiration and dispense,Blowout before,1,89.167931
3,Viscosity_std_1275,rLine1000,1000.0,36.143918,36.143918,False,10,10,0,0.8736,119.593486,0.612891,-29.843014,Incomplete aspiration and dispense,Blowout before,2,75.334345
4,Viscosity_std_1275,rLine1000,500.0,36.143918,36.143918,False,10,10,0,0.8736,91.498749,0.369757,-15.348596,Incomplete aspiration and dispense,Blowout before,2,75.334345
5,Viscosity_std_1275,rLine1000,300.0,36.143918,36.143918,False,10,10,0,0.8736,80.366316,0.200310,-23.569327,Incomplete aspiration and dispense,Blowout before,2,75.334345
6,Viscosity_std_1275,rLine1000,1000.0,2.891513,36.143918,False,10,10,0,0.8736,436.806757,0.746786,-14.516293,Incomplete aspiration and dispense,Blowout before,3,393.506829
7,Viscosity_std_1275,rLine1000,500.0,2.891513,36.143918,False,10,10,0,0.8736,250.100245,0.383848,-12.122796,Incomplete aspiration and dispense,Blowout before,3,393.506829
8,Viscosity_std_1275,rLine1000,300.0,2.891513,36.143918,False,10,10,0,0.8736,175.548413,0.232206,-11.398909,Incomplete aspiration and dispense,Blowout before,3,393.506829
9,Viscosity_std_1275,rLine1000,1000.0,36.143918,2.891513,False,10,10,0,0.8736,437.120528,0.761357,-12.848350,Incomplete aspiration,Blowout before,4,393.506829


In [9]:
liquids_dict = {
  liquid_name :{
        "rLine1000": {
            "aspiration_rate": df_para.loc[1,['aspiration_rate']], 
            "dispense_rate": df_para.loc[1,['dispense_rate']], 
            "blow_out" : False, 
            "delay_aspirate" : 10, 
            "delay_dispense" : 10, 
            "delay_blow_out" : 0, 
            },
    }

}

In [15]:
liquids_dict[liquid_name][pipette_name]['dispense_rate']

dispense_rate    28.915134
Name: 1, dtype: object

Running the experiment

In [ ]:
def loop_pipetting(iterations, volume, liquid_level):
    import pandas as pd
    counter = 1
    pipette.mover.setHandedness(False)

    while counter <= iterations:
        #Transfer start
        start = time.time()

        #Aspirate step
        pipette.mover.safeMoveTo(source.wells['A1'].from_bottom((0,0,liquid_level-5)))
        pipette.liquid.aspirate(volume, speed=liquids_dict[liquid_name][pipette_name]['aspiration_rate'])
        time.sleep(liquids_dict[liquid_name][pipette_name]['delay_aspirate'])

        pipette.touchTip(source.wells['A1'])

        #Dispense step
        pipette.mover.safeMoveTo(balance_deck.wells['A1'].from_top((0,0,-5)))

        balance.tare()
        balance.clearCache()
        balance.toggleRecord(True)
        time.sleep(5)


        pipette.liquid.dispense(volume, speed=liquids_dict[liquid_name][pipette_name]['dispense_rate'])

        time.sleep(liquids_dict[liquid_name][pipette_name]['delay_dispense'])

        #Blowout step
        if liquids_dict[liquid_name][pipette_name]['blow_out'] == True:
            pipette.liquid.blowout(home=False)
            time.sleep(liquids_dict[liquid_name][pipette_name]['delay_blow_out'])

        #Transfer termination
        finish = time.time()
        time_m = finish - start

        pipette.mover.safeMoveTo(source.wells['A1'].top)
        time.sleep(5)
        balance.toggleRecord(False)
        if liquids_dict[liquid_name][pipette_name]['blow_out'] == True:
            pipette.liquid.home()

        #Do blowout
        pipette.liquid.blowout(home=False)
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()
        time.sleep(5)
        pipette.liquid.blowout(home=False)
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()
        time.sleep(5)
        pipette.liquid.blowout(home=False)
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()

        #Record transfer values 
        m = (balance.buffer_df.iloc[-10:,-1].mean()-balance.buffer_df.iloc[:10,-1].mean())/1000

        #Update dataframe
        df = pd.concat([df,pd.DataFrame(liquids_dict[liquid_name][pipette_name],index=[0])],ignore_index=True)
        error = (m-density*volume/1000)/(density/1000*volume)*100
        df.iloc[-1,-5] = time_m
        df.iloc[-1,2] = volume
        df.iloc[-1, 0] = liquid_name
        df.iloc[-1, 1] = pipette_name
        df.iloc[-1,-6] = density
        df.iloc[-1, -4] = m
        df.iloc[-1,-3]= error

        #Check liquid levels
        liquid_level = liquid_level - 1.2*m/density

        #Liquid level checks
        if (1.2*m/density > 1) and (1.2*m/density < 0):
            break
        if (liquid_level > initial_liquid_level) and (liquid_level < 5):
            break

        # Increment Tries
        counter += 1

In [ ]:
#Check if new tip is required
pipette.mover.setSpeed(50)
pipette.mover.setHandedness(False)

if pipette.liquid.isTipOn()== False:
    pipette.attachTip()

#setup for for loop
volumes_list = [1000,500,300]
liquid_level = initial_liquid_level

for item in volumes_list:

    #New dataframe
    df = pd.DataFrame(columns = ['liquid', 'pipette', 'volume', 'aspiration_rate', 'dispense_rate','blow_out', 'delay_aspirate', 'delay_dispense','delay_blow_out', 'density', 'time', 'm', '%error'])
    df = df.astype({'liquid':str,'pipette':str,'blow_out':bool,'Transfer_Observation':str,'Comment':str})

    #run liquid transfer function
    loop_pipetting(iterations=10, volume=item, liquid_level=liquid_level)



In [ ]:
#save after each standard-experiment iteration
df.to_csv(liquid_name+'_optimization_exp3.csv', index = False)